# Deconvolution Example
In this notebook, a small example is shown on how to use the deconvolution classes. Deconvolution is a method used to remove the effect of the multiple scattering on the experimental EEL spectra. Two different deconvolution methods have been implemented:

1. Gaussian Modifier: Deconvolution of the spectrum using the low loss and doing conventional deconvolution via the Fast Fourier Transform. A Gaussian is used to damp the higher frequencies (loss of energy resolution). 
   
2. Richardson Lucy: Based on maximum likelihood deconvolution with the assumption of Poissonian distributed noise 

See [Verbeeck et al.](https://doi.org/10.1016/j.ultramic.2009.06.010) for a more in depth description on the two methods. In the philisophy of EELSMODEL, deconvolution should be avoided as much as possible and the multiple scattering should be accounted for in the model itself.

In [1]:
%matplotlib qt

In [18]:
import pyEELSMODEL.api as em
from pyEELSMODEL.components.fixedpattern import FixedPattern
import numpy as np
import os
import matplotlib.pyplot as plt

## Simulation 
This part simualtes a core-loss STEM-EELS spectrum on Ce 3+. The thickness varies in different part of the map. *This section only performs the STEM-EELS simulation hence it is not very important for the deconvolution part*

In [6]:
def make_circular_mask(xco, yco, Rin, Rout, shape):
    XX, YY = np.meshgrid(np.arange(shape[1])-yco, np.arange(shape[0])-xco)
    
    R = np.sqrt(XX**2+YY**2)
    
    mask = np.zeros(shape)
    boolean = (R>=Rin) & (R<Rout)
    mask[boolean] = 1
    return mask

def make_rectangular_mask(xco, yco, width,height, shape):
    mask = np.zeros(shape)
    mask[xco:xco+width, yco:yco+height] = 1
    return mask
    

In [10]:
xsize = 64
ysize = 64
maps = np.zeros((1,xsize,ysize))


mask0 =make_rectangular_mask(3, 3, 20, 20, (xsize,ysize))
mask1 =  make_rectangular_mask(50, 50, 10, 10, (xsize,ysize))
mask2 = make_circular_mask(xsize//2, ysize//2, 0, 10, (xsize,ysize))

maps[0] = 1 + 2*mask0 + 3*mask1 + 4*mask2

Load an artificial Ce M edge with fine structure. 

In [12]:
file_ce3 = os.path.join(os.getcwd(), r'data\ce3_edge.hdf5')
ce3 = em.Spectrum.load(file_ce3)

In [16]:
tlambda_map = np.zeros_like(adf)
for ii in range(maps.shape[0]):
    tlambda_map += 1*maps[ii]

# the maximal inelastic mean free path is 0.5
tlambda_map = 0.5*tlambda_map/tlambda_map.max()

fig, ax = plt.subplots()
ax.imshow(tlambda_map, cmap='gray')


In [22]:
settings = (300e3, 1e-9, 20e-3)
msh = em.MultiSpectrumshape(0.2, 840, 1024, xsize, ysize)
sh = msh.getspectrumshape()

In [23]:
sim = em.CoreLossSimulator(msh, [], [], maps, tlambda_map, settings)
sim.fwhm=0.5 #resolution of zlp
sim.bg_method = 'powerlaw' #background model
sim.noise = 1e3 #this reduces the noise by factor sqrt(noise) 
sim.make_lowloss() 

sim.element_components = []
sim.element_components.append(FixedPattern(sh, ce3)) #add Ce3+ coreloss edge for simulation
sim.make_coreloss()


4096it [00:02, 1557.67it/s]
4096it [00:02, 1383.15it/s]


In [24]:
hl = sim.multispectrum
ll = sim.ll

## Deconvolution
In this part , the deconvolution is performed on a single and a multispectrum using the two methods. Before deconvolving, the background is removed by fitting a power-law to the region before the edge onset. 

#### Background removal

In [34]:
signal_ranges = [840,870]
back = em.BackgroundRemoval(hl, signal_ranges)
rem = back.calculate_multi()

True


4096it [00:00, 7139.83it/s]
4096it [00:05, 709.15it/s]
4096it [00:00, 21436.89it/s]


#### Deconvolution on single spectrum


In [ ]:
from pyEELSMODEL.operators.deconvolutions.richardson_lucy import RichardsonLucyDeconvolution
from pyEELSMODEL.operators.deconvolutions.gaussianmodifier import GaussianModifier

In [27]:
indx=32
indy=32

s = rem[indx, indy]
l =ll[indx, indy]

In [32]:
rich = RichardsonLucyDeconvolution(s, l, iterations=3)
logr = GaussianModifier(s, l, factor=None) #if none is chosen it use the fwhm of the low loss to get esitmate gaussian fwhm

s_r = rich.deconvolve()
s_l = logr.deconvolve()

In [33]:
win = (800,1000) #normalization region in eV. 
fig, ax = plt.subplots()
ax.plot(s.energy_axis, s.data/s.integrate(win), label='Raw spectrum')
ax.plot(s.energy_axis, s_r.data/s_r.integrate(win), label='RL deconvolution')
ax.plot(s.energy_axis, s_l.data/s_l.integrate(win), label='Gaussian modifier')
ax.plot(s.energy_axis, sim.element_components[0].data/sim.element_components[0].integrate(win), label='Single scattering cross section')
ax.legend()

#### Deconvolution on multispectrum

In [30]:
rich = RichardsonLucyDeconvolution(rem, ll)
logr = GaussianModifier(rem, ll, factor=4)


rem_r = rich.deconvolve()
rem_l = logr.deconvolve()

4096it [00:07, 570.27it/s]
4096it [00:01, 3724.96it/s]


In [31]:
labels= ['Raw spectrum', 'RL deconvolution', 'Gaussian modifier']
em.MultiSpectrumVisualizer([rem, rem_r, rem_l], labels=labels)